# Analyzing Performance at the 2025 SF Marathon: Terrain, Pace, and Heart Rate

This notebook explores my performance at the 2025 San Francisco Marathon. The data is pulled from Strava records of my 26.2-mile race. The focus of this exploratory analysis is to identify the relationship between course structure and terrain (elevation, distance segments, and aid stations) on performance metrics such as pace and heartrate. The goal is to draw insights to guide future training and serve as a reference if I run this marathon agian. 

The analysis is done using SQL and Python (pandas, matplotlib) in a Jupyter notebook. Raw data (TCX and GPX) was cleaned in separate Python scripts. 

## 1. Overview of the Data 

In [1]:
# Data, SQL, visualization imports
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go


KeyboardInterrupt: 

In [ ]:
# Connect to SQLite database 
con = sqlite3.connect("../data/marathon_data.db") 
cur = con.cursor()

# Table overview
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
print("Tables:", tables['name'].tolist())

# Show structure for each table
for table in tables['name']:
    print(f"\n=== {table.upper()} ===")
    cols = pd.read_sql(f"PRAGMA table_info({table});", con)
    for _, row in cols.iterrows():
        print(f"  {row['name']}: {row['type']}")

The database contains three tables: `tcx_data`, `official_route_data`, and `aid_stations_data`. The names and data types of the columns in each are as follows:

- `tcx_data`: data from my race recorded on my Apple Watch
    - __time__: timestamp in UTC for each row entry; TEXT (convertible to date-time)
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __elevation__: altitude above sea level, in meters; REAL
    - __distance__: cumulative distance covered since the start, in meters; REAL
    - __heart_rate__: runner’s heart rate at that moment, in beats per minute (BPM); INTEGER

- `official_route_data`: official route data provided by SFM organizers
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __elevation__: altitude above sea level, in meters; REAL

- `aid_stations_data`: official route data provided by SFM organizers
    - __name__: name of the waypoint; TEXT
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __type__: type of the waypoint; TEXT

The following cell shows the first 5 rows of each table. 

In [ ]:
# Sample data from each table
tables_to_show = [
    ("tcx_data", "TCX Data"),
    ("official_route_data", "Official Route Data"),
    ("aid_stations_data", "Aid Stations Data")
]

for table, name in tables_to_show:
    print(f"=== {name} (First 5 Rows) ===")
    sample = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", con)
    display(sample)
    print() 
    

## 2. Data cleaning and processing

Before performing any analysis, the raw data is inspected for anomalies to ensure accurate results. 

- **Distance units:** Original distance values were in meters; they are converted to kilometers for readability.
- **Pace calculation:** Pace was computed in a SQL view prior to loading into a pandas dataframe for efficiency.
- **Route validation:** To confirm data quality, the elevation profile is compared against the official course route published by the organizers on Strava. This ensures the cleaned data aligns with the actual course.
- **Distance calculation for the official course data:** The original data only contains coordinates and elevation; distance in km was calculated in Python to allow smoother comparison with my data. 
- **Elevation spikes:** One elevation measurement (~250 m) was clearly outside the plausible range for the course. This was replaced using linear interpolation to preserve continuity.


In [ ]:
# Create a view for processed tcx race data with pace and distance in km
race_query = """
CREATE VIEW IF NOT EXISTS processed_view AS
SELECT
    time,
    latitude,
    longitude,
    elevation,
    distance / 1000.0 AS distance_km,
    heart_rate,
    -- compute delta time (sec)
    CAST(
        (strftime('%s', time) - strftime('%s', LAG(time) OVER (ORDER BY time)))
        AS REAL
    ) AS delta_time,
    -- compute delta distance (m)
    (distance - LAG(distance) OVER (ORDER BY time)) AS delta_dist,
    -- compute pace (min/km)
    CAST(
        (strftime('%s', time) - strftime('%s', LAG(time) OVER (ORDER BY time))) / 60.0 /
        NULLIF((distance - LAG(distance) OVER (ORDER BY time)) / 1000.0, 0)
        AS REAL
    ) AS pace
FROM tcx_data
"""

cur.execute(race_query)
con.commit()

# Load view to pandas dataframe
tcx_df = pd.read_sql("SELECT * FROM processed_view;", con)

In [ ]:
# Query official route data
official_route_query = """
SELECT
    latitude,
    longitude,
    elevation
FROM official_route_data
"""

cur.execute(official_route_query)
con.commit()

# Create a dataframe to store official route data with segment distance and distance from start in km
official_route_df = pd.read_sql("SELECT * FROM official_route_data;", con)

distances = [0.0]
for i in range(1, len(official_route_df)):
    start = (official_route_df.iloc[i-1]['latitude'], official_route_df.iloc[i-1]['longitude'])
    end = (official_route_df.iloc[i]['latitude'], official_route_df.iloc[i]['longitude'])
    distances.append(geodesic(start, end).km)

official_route_df['segment_distance'] = distances
official_route_df['cumulative_distance_km'] = official_route_df['segment_distance'].cumsum()


In [ ]:
# Query aid stations data
aid_stations_query = """
SELECT
    latitude,
    longitude,
    name
FROM aid_stations_data
"""

cur.execute(aid_stations_query)
con.commit()

# Create a dataframe to store aid stations data
aid_stations_df = pd.read_sql("SELECT * FROM aid_stations_data;", con)


In [ ]:
# Create subplots for better visibility
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Elevation Comparison', 'Route Map Comparison'),
    vertical_spacing=0.15,
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}]]
)

# First subplot: Elevation comparison
fig.add_trace(
    go.Scatter(
        x=tcx_df['distance_km'],
        y=tcx_df['elevation'],
        mode='lines',
        name='My Race Elevation',
        line=dict(color='blue', width=2),
        opacity=0.7,
        hovertemplate='Distance: %{x:.2f} km<br>My Elevation: %{y:.1f} m<extra></extra>'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=official_route_df['cumulative_distance_km'],
        y=official_route_df['elevation'],
        mode='lines',
        name='Official Route Elevation',
        line=dict(color='red', width=2),
        opacity=0.7,
        hovertemplate='Distance: %{x:.2f} km<br>Official Elevation: %{y:.1f} m<extra></extra>'
    ),
    row=1, col=1
)

# Second subplot: Route map
fig.add_trace(
    go.Scatter(
        x=tcx_df['longitude'],
        y=tcx_df['latitude'],
        mode='lines',
        name='My Race Route',
        line=dict(color='blue', width=3),
        opacity=0.7,
        hovertemplate='Lat: %{y:.5f}<br>Lon: %{x:.5f}<br>My Route<extra></extra>',
        showlegend=False  # Don't duplicate legend
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=official_route_df['longitude'],
        y=official_route_df['latitude'],
        mode='lines',
        name='Official Route',
        line=dict(color='red', width=2, dash='dash'),
        opacity=0.7,
        hovertemplate='Lat: %{y:.5f}<br>Lon: %{x:.5f}<br>Official Route<extra></extra>',
        showlegend=False  # Don't duplicate legend
    ),
    row=2, col=1
)

# Add aid stations to map
fig.add_trace(
    go.Scatter(
        x=aid_stations_df['longitude'],
        y=aid_stations_df['latitude'],
        mode='markers',
        name='Aid Stations',
        marker=dict(color='green', size=8, symbol='circle'),
        hovertemplate='%{text}<br>Lat: %{y:.5f}<br>Lon: %{x:.5f}<extra></extra>',
        text=aid_stations_df['name'] if 'name' in aid_stations_df.columns else 'Aid Station',
        showlegend=False  # Don't duplicate legend
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title_text='Marathon Analysis: Elevation and Route Comparison',
    height=900,
    width=900,
    showlegend=True
)

# Update x-axis labels
fig.update_xaxes(title_text="Distance (km)", row=1, col=1)
fig.update_xaxes(title_text="Longitude", row=2, col=1)

# Update y-axis labels
fig.update_yaxes(title_text="Elevation (m)", row=1, col=1)
fig.update_yaxes(title_text="Latitude", row=2, col=1, scaleanchor="x2", scaleratio=1)

fig.show()

In [ ]:
# Replace obvious outlier with interpolation
tcx_df.loc[tcx_df['elevation'] > 100, 'elevation'] = None
tcx_df['elevation'] = tcx_df['elevation'].interpolate()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['elevation'],
    mode='lines',
    name='Elevation',
    line=dict(color='blue', width=2),
    hovertemplate='Distance: %{x:.2f} km<br>Elevation: %{y:.1f} m<extra></extra>'
))

fig.update_layout(
    title='Elevation over Full Marathon Route',
    xaxis_title='Distance (km)',
    yaxis_title='Elevation (m)',
    width=900,
    height=500,
    hovermode='x'
)

fig.show()


## 3. Overall Trends

The following graph illustrates my pace and elevation across the 26.2-mile race. 

In [ ]:
overall_trend_query = """
SELECT distance / 1000.0 AS distance_km, pace, elevation
FROM pace_view
ORDER BY distance
"""
df = pd.read_sql_query(overall_trend_query, con)

def format_pace(pace):
    if pace is None or np.isnan(pace):
        return None
    minutes = int(pace)
    seconds = int(round((pace - minutes) * 60))
    return f"{minutes}:{seconds:02d}"

df['pace_str'] = df['pace'].apply(format_pace)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['distance_km'],
    y=df['pace'],
    mode='lines',
    name='Pace (min/km)',
    line=dict(color='blue'),
    customdata=df['pace_str'],  # attach formatted pace
    hovertemplate='Distance: %{x:.2f} km<br>Pace: %{customdata} min/km<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df['distance_km'],
    y=df['elevation'],
    mode='lines',
    name='Elevation (m)',
    line=dict(color='red'),
    yaxis='y2',
    hovertemplate='Distance: %{x:.2f} km<br>Elevation: %{y} m<extra></extra>'
))

fig.update_layout(
    title='Pace and Elevation over Distance',
    xaxis_title='Distance (km)',
    yaxis=dict(title='Pace (min/km)', side='left'),
    yaxis2=dict(title='Elevation (m)', overlaying='y', side='right')
)

fig.show()




## . Heart Rate vs Pace Analyis

## . Time Losses at Events

## . Heatmap

In [ ]:
con.commit()
con.close()


## . Conclusions